In [16]:
library('tidyverse')
library('tidylog')
library('sf')
library('mapview')
library('googlePolylines')


Attaching package: ‘tidylog’


The following objects are masked from ‘package:dplyr’:

    add_count, add_tally, anti_join, count, distinct, distinct_all,
    distinct_at, distinct_if, filter, filter_all, filter_at, filter_if,
    full_join, group_by, group_by_all, group_by_at, group_by_if,
    inner_join, left_join, mutate, mutate_all, mutate_at, mutate_if,
    relocate, rename, rename_all, rename_at, rename_if, rename_with,
    right_join, sample_frac, sample_n, select, select_all, select_at,
    select_if, semi_join, slice, slice_head, slice_max, slice_min,
    slice_sample, slice_tail, summarise, summarise_all, summarise_at,
    summarise_if, summarize, summarize_all, summarize_at, summarize_if,
    tally, top_frac, top_n, transmute, transmute_all, transmute_at,
    transmute_if, ungroup


The following objects are masked from ‘package:tidyr’:

    drop_na, fill, gather, pivot_longer, pivot_wider, replace_na,
    spread, uncount


The following object is masked from ‘package:stats

In [2]:
# Aplicar funcoes em paralelo
library('future.apply')
# Checando: Jupyter suporta multicore?
future::supportsMulticore()

Loading required package: future



[1] TRUE

In [3]:
# https://stackoverflow.com/questions/40536067/how-to-adjust-future-global-maxsize
# For 850MB: 850*1024^2 = 891289600
# For +1.3GB: 1500*1024^2 = 1572864000
# options(future.globals.maxSize = 891289600)
options(future.globals.maxSize = 5e6) # 5 MB
options(future.globals.maxSize = 285*1024^2) # 285 MB
parallelly::availableCores()

system 
     8

In [4]:
# Estrutura de pastas
pasta_dados        <- "../../yellow_dados"
pasta_osm_sp       <- sprintf("%s/02_osm_simplificado_sp", pasta_dados)
pasta_map_matching <- sprintf("%s/05_map_matching", pasta_dados)
pasta_graphhopper  <- sprintf("%s/07_graphhopper", pasta_dados)
pasta_gh_pbfs      <- sprintf("%s/03_PBFs_SP_rede_2019", pasta_graphhopper)
pasta_orig_vs_mod  <- sprintf('%s/10_rotas_originais_vs_modeladas', pasta_dados)
pasta_orig_way_ids <- sprintf("%s/01_osm_way_ids_rotas_modeladas", pasta_orig_vs_mod)
pasta_rotas_mod    <- sprintf("%s/02_rotas_modeladas", pasta_orig_vs_mod)
dir.create(pasta_rotas_mod, recursive = TRUE, showWarnings = FALSE)

In [5]:
# ------------------------------------------------------------------------------
# Funções
# ------------------------------------------------------------------------------

# Transforma linha de polyline para dataframe com latlongs
polyline_to_latlong <- function(polyline, trip_id){
  # polyline <- viagem$poly; trip_id <- viagem$trip_id
  this <- as.data.frame(decode(as.character(polyline)))
  this <- this %>% 
    # Formatação dos pontos está fora do lugar: 
    # de -235.641 para -23.5641 - ajeitar
    mutate(trip_id  = trip_id,
           lat = str_replace(lat, '\\.', ''),
           lon = str_replace(lon, '\\.', ''),
           lat = as.double(str_replace(lat, '-23', '-23.')),
           lon = as.double(str_replace(lon, '-46', '-46.')))
  
  return(this)
}


# Transforma dataframe com várias linhas de latlon em sf
df_latlong_to_sf <- function(df, trip_id, st_type = 'LINESTRING'){
  # df <- this
  this <- df %>% 
    # Transformar em sf
    st_as_sf(coords = c("lon", "lat"), crs = 4326) %>% 
    # Transformar pontos em linha - ver possíveis erros em
    # https://github.com/r-spatial/sf/issues/321
    # # Modo 1 - Com st_coordinates, retorna matriz
    # Retrieve coordinates in matrix form 
    # st_coordinates() %>%
    # st_linestring()
    # Modo 2 - Com summarize, retorna sf
    # Aqui, o summarize pode ser qualquer coisa, o 
    # importante é o 'do_union=FALSE'
    group_by(trip_id) %>% 
    summarize(m = n(), do_union = FALSE) %>% 
    select(-m) %>% 
    st_cast(st_type)
  
  return(this)
}


# Insere distâncias calculadas em vias comuns ou infra cicloviária por linha do df original
calcular_dist_ciclo <- function(tripid) {
  # line <- rotas_m %>% slice(1)
  line <- rotas_m %>% filter(trip_id == tripid)
  
  # Puxar osm_ids da rota modelada
  trip <- list.files(pasta_orig_way_ids, pattern = line$trip_id, recursive = FALSE, full.names = TRUE)
  trip <- read_delim(trip, delim = ';', col_types = 'cc')
  # trip_way_ids <- paste0("'", trip_id$osm_way_id, "'", collapse = ",")
  
  # Filtrar osm_ids da rota modelada do viário completo de SP
  viario_rota <- viario_sp %>% filter(osm_id %in% trip$osm_way_id)
  # mapview(viario_rota)
  
  # Transformar o polyline da rota modelada em shapefile
  shape_rota <- polyline_to_latlong(line$poly, line$trip_id)
  shape_rota <- df_latlong_to_sf(shape_rota, line$trip_id)
  shape_rota <- shape_rota %>% st_transform(31983) %>% mutate(dist = round(st_length(.), 4))
  # mapview(shape_rota)
  
  
  # ----------------------------------------------------------------------------
  # Calcular detour
  # ----------------------------------------------------------------------------
  
  # Adicionar extensão da linha do shapefile da rota modelada e calcular detour
  line <- line %>% mutate(dist_linha_mod = as.double(shape_rota$dist),
                          detour_mod = dist_linha_mod / dist_reta, 
                          .after = 'detour_mm')
  # line %>% select(trip_id, dist_total, dist_reta, dist_linha, detour_mm, dist_linha_mod, detour_mod)
  
  # ----------------------------------------------------------------------------
  # Calcular extensões percorridas em vias comuns e infra cicloviária
  # ----------------------------------------------------------------------------
  
  # A partir de um pequeno buffer criado no polyline da rota modelada, fazer uma 
  # interseção nos osm_ids originais - isso porque os osm_ids podem ter várias 
  # quadras e o segmento percorrido ser só um trechinho dele
  buffer_rota <- st_buffer(shape_rota, 2)
  viario_rota_cropped <- st_intersection(viario_rota, buffer_rota)
  # mapview(viario_rota_cropped) + mapview(buffer_rota)
  
  # Recaulcular as extensões dos arcos (no caso, as extensões percorridas dentro
  # daquele osm_id), transformar em dataframe e isolar colunas de interesse
  viario_rota_cropped <- viario_rota_cropped %>% mutate(new_ext = as.double(st_length(.)), .after = 'length_m')
  viario_rota_cropped <- viario_rota_cropped %>% st_drop_geometry() %>% select(osm_id, new_ext)
  
  # Juntar com dados da infraestrutura cicloviária
  viario_rota_cropped <- 
    viario_rota_cropped %>% 
    left_join(infra_ciclo, by = 'osm_id') %>% 
    mutate(infra_ciclo = ifelse(is.na(infra_ciclo), 'via_comum', infra_ciclo))
  
  # Fator de ajuste para as distâncias - vamos aplicar um proporcional geral a partir
  # da diferença entre a extensão total da rota modelada e a calculada agora
  fator_correcao <- line$distance / sum(viario_rota_cropped$new_ext)
  viario_rota_cropped <- viario_rota_cropped %>% mutate(ext_rev = new_ext * fator_correcao)
  
  # Agrupar extensões por tipo de viário percorrido
  viario_rota_cropped <- 
    viario_rota_cropped %>% 
    group_by(infra_ciclo) %>% 
    summarise(ext = sum(ext_rev)) %>% 
    ungroup() %>% 
    mutate(trip_id = line$trip_id) %>% 
    pivot_wider(id_cols = trip_id,
                names_from = 'infra_ciclo',
                values_from = ext)
  
  # Checar se todas as colunas de tipo de viário estão como colunas - se não, inserir
  for (i in c('ciclo_expressa', 'ciclo_comum', 'ciclofaixa', 'via_comum')) {
    if (!i %in% names(viario_rota_cropped)) {
      # Inserir nova coluna como NA (NA_real_, NA_character_)
      viario_rota_cropped <- viario_rota_cropped %>% mutate(!!i := 0)
    }
    
  }
  
  # Somar extensões percorridas em infra cicloviária
  viario_rota_cropped <- viario_rota_cropped %>% mutate(infra_ciclo = ciclo_expressa + ciclo_comum + ciclofaixa,
                                                        .after = 'via_comum')
  
  # Juntar todas as infos ao dataframe original e reordenar colunas
  trip_out <- 
    line %>% 
    left_join(viario_rota_cropped, by = 'trip_id') %>% 
    relocate(c(via_comum, infra_ciclo, ciclo_expressa, ciclo_comum, ciclofaixa), .before = 'vg_contramao')
  
  
  # ----------------------------------------------------------------------------
  # Calcular acurácia da rota modelada - por pontos
  # ----------------------------------------------------------------------------
  
  # Aplicar buffer de 50 metros - a distância está sendo definida pela 
  # literatura, mas poderia ser menor até
  buffer <- st_buffer(shape_rota, 50)
  # mapview(buffer)
  
  # Abrir arquivo com latlon da viagem original - as coordenadas que serão
  # consideradas são as resultantes do map matching
  that <- mm_files %>% filter(str_detect(arq, line$trip_id)) %>% pull()
  that <- read_delim(that, delim = ';', col_types = "cidddddddicddcccddd")
  that <- that %>% select(trip_id, lat = matched_points.lat, lon = matched_points.lon)
  that <- that %>% distinct()
  that <- df_latlong_to_sf(that, trip, st_type = 'POINT')
  that <- st_transform(that, 31983) %>% add_column(idx = 1:nrow(.))
  # mapview(that, cex = 3, zcol = 'idx') + mapview(buffer)
  
  # Quantos pontos do map matching estão dentro do buffer considerado?
  those <- filter(that, st_intersects(that, buffer, sparse = FALSE))
  # mapview(those)
  
  # Guardar resultados em um dataframe
  resultado <- data.frame(trip_id      = line$trip_id,
                          pts_intsct   = nrow(those),
                          pts_viagem   = nrow(that),
                          acuracia_pontos  = nrow(those) / nrow(that) * 100)
  
  # Juntar todas as infos ao dataframe original e reordenar colunas
  trip_out <- 
    trip_out %>% 
    left_join(resultado, by = 'trip_id') %>% 
    relocate(c(pts_intsct, pts_viagem, acuracia_pontos), .after = 'speed')
  
  
  # ----------------------------------------------------------------------------
  # Calcular acurácia da rota modelada - por extensão
  # ----------------------------------------------------------------------------
  
  # Transformar pontos da rota original (map matching) em linestring
  this <- df_latlong_to_sf(that, line$trip_id)
  # mapview(this)
  
  # Quanto das linhas da rota original está dentro do buffer considerado?
  thou <- st_intersection(this, buffer)
  # mapview(thou) + mapview(buffer)
  
  # Calcular extensão dos trechos que estão dentro do buffer
  thou <- thou %>% mutate(ext_linhas = as.double(st_length(.)))
  thou <- thou %>% st_drop_geometry() %>% select(trip_id = trip_id.1 , ext_linhas)
  
  # Calcular acurácia das linhas - se a extensão das linhas for maior do que
  # a distância da rota, acurácia vai ser 100%
  thou <- thou %>% mutate(acuracia_linhas = ifelse(line$distance >= ext_linhas,
                                                   ext_linhas / line$distance * 100,
                                                   100))
  
  
  # Juntar todas as infos ao dataframe original e reordenar colunas
  trip_out <- 
    trip_out %>% 
    left_join(thou, by = 'trip_id') %>% 
    relocate(c(ext_linhas, acuracia_linhas), .after = 'speed')
  
  # Gravar resultados
  out_file <- sprintf('%s/%s_modelado.csv', pasta_rotas_mod, tripid)
  write_delim(trip_out, out_file, delim = ';')
  
}

In [6]:
# ------------------------------------------------------------------------------
# Rede cicloviária 2019
# ------------------------------------------------------------------------------

# Abrir arquivo com osm_ids de ciclovias expressas em 2019 - a classificação que
# está no dataframe de atributos do viário, acima, contém osm_ids das vias 
# próximas, já que o map matching foi feito no modo 'pedestrian' e ao passar
# por essas vias, o osm_id que ia ser considerado era o do viário
ciclo_expressas <- sprintf('%s/00_atributos_ciclovias_expressas_2019.csv', pasta_gh_pbfs)
ciclo_expressas <- read_delim(ciclo_expressas, delim = ';', col_types = 'c') %>% distinct()
ciclo_expressas <- ciclo_expressas %>% mutate(infra_ciclo = 'ciclo_expressa')

# Abrir o arquivo com osm_ids de vias com ciclofaixa em 2019
ciclo_ciclofx <- sprintf('%s/02_atributos_ciclofaixas_lcn.csv', pasta_gh_pbfs)
ciclo_ciclofx <- read_delim(ciclo_ciclofx, delim = ';', col_types = 'c') %>% distinct()
ciclo_ciclofx <- ciclo_ciclofx %>% mutate(infra_ciclo = 'ciclofaixa')

# Abrir o arquivo com osm_ids de ciclovias comuns (não expressas) em 2019
ciclo_comuns <- sprintf('%s/01_atributos_ciclovias_comuns_2019.csv', pasta_gh_pbfs)
ciclo_comuns <- read_delim(ciclo_comuns, delim = ';', col_types = 'c') %>% distinct()

# Abrir arquivo de trechos de ciclovias comuns em que não há semáforos ou interseções
ciclo_ciclov_semsem <- sprintf('%s/03_atributos_ciclovias_comuns_sem_semaforo.csv', pasta_gh_pbfs)
ciclo_ciclov_semsem <- read_delim(ciclo_ciclov_semsem, delim = ';', col_types = 'c') %>% distinct()

# Ciclovias comuns serão ciclo_comuns + ciclovias sem semáforos ou interseções
ciclo_comuns <- rbind(ciclo_comuns, ciclo_ciclov_semsem)
ciclo_comuns <- ciclo_comuns %>% mutate(infra_ciclo = 'ciclo_comum')


# Juntar tudo em um único dataframe
infra_ciclo <- rbind(ciclo_expressas, ciclo_ciclofx, ciclo_comuns)
rm(ciclo_expressas, ciclo_ciclofx, ciclo_comuns, ciclo_ciclov_semsem)


# ------------------------------------------------------------------------------
# Bases de dados a serem utilizadas
# ------------------------------------------------------------------------------

# Puxar listagem de viagens originais (latlon virá do map matching)
pasta_mm_1 <- sprintf('%s/201811/viagens_processadas_csv', pasta_map_matching)
pasta_mm_2 <- sprintf('%s/201812/viagens_processadas_csv', pasta_map_matching)
pasta_mm_3 <- sprintf('%s/201901/viagens_processadas_csv', pasta_map_matching)
mm_files1 <- list.files(pasta_mm_1, pattern = '^([0-9]{6})_([0-9]{2}).csv', recursive = FALSE, full.names = TRUE) %>% as.data.frame()
mm_files2 <- list.files(pasta_mm_2, pattern = '^([0-9]{6})_([0-9]{2}).csv', recursive = FALSE, full.names = TRUE) %>% as.data.frame()
mm_files3 <- list.files(pasta_mm_3, pattern = '^([0-9]{6})_([0-9]{2}).csv', recursive = FALSE, full.names = TRUE) %>% as.data.frame()
mm_files <- rbind(mm_files1, mm_files2, mm_files3) %>% rename(arq = '.')
rm(mm_files1, mm_files2, mm_files3, pasta_mm_1, pasta_mm_2, pasta_mm_3)
# head(mm_files)


# Abrir origens e destinos das rotas iniciais da Yellow - são consideradas aqui
# (pelos scripts anteriores) as rotas que (a) tiveram algum trecho considerado
# no modelo; (b) que não foram divididas em trechos menores; e (c) em que o
# trecho único considerado é o inicial (possui trip_id com _00)
ods_vgs <- sprintf('%s/03_rotas_originais_infraciclo_detour_carac_viagens.csv', pasta_orig_vs_mod)
ods_vgs <- read_delim(ods_vgs, delim = ';', col_types = 'cccdddddccccdddddddd')
# Remover viagens com origem e destino no mesmo qgis_id
ods_vgs <- ods_vgs %>% filter(qgis_id.x != qgis_id.y)
# Remover colunas de uso de infraestrutura cicloviária das rotas originais (map
# matching). Dados de latlong já vão estar presentes nas rotas modeladas
ods_vgs <- ods_vgs %>% select(-c(via_comum, infra_ciclo, ciclo_expressa, ciclo_comum, ciclofaixa,
                                 lat.x, lon.x, lat.y, lon.y))
head(ods_vgs)


# Abrir viário de SP com osm_ids
viario_sp <- read_sf(sprintf('%s/sao_paulo_osm_filtrado_com_qgis_id.gpkg', pasta_osm_sp))
# Nos interessam os osm_ids e, talvez, a extensão original dos arcos da rede
viario_sp <- viario_sp %>% select(osm_id, length_m)
head(viario_sp)

distinct: removed 19 rows (46%), 22 rows remaining

mutate: new variable 'infra_ciclo' (character) with one unique value and 0% NA

distinct: removed 64 rows (2%), 2,767 rows remaining

mutate: new variable 'infra_ciclo' (character) with one unique value and 0% NA

distinct: removed 29 rows (4%), 768 rows remaining

distinct: no rows removed

mutate: new variable 'infra_ciclo' (character) with one unique value and 0% NA

rename: renamed one variable (arq)

filter: no rows removed

select: dropped 9 variables (via_comum, infra_ciclo, ciclo_expressa, ciclo_comum, ciclofaixa, …)



trip_id,qgis_id.x,qgis_id.y,vg_contramao,vg_loop,vg_exper,vg_parques,dist_total,dist_reta,dist_linha,detour_mm
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
000135_00,271157,104168,não,não,não,não,821.4461,562.5676,802.6903,1.426834
000138_00,161929,161977,não,não,não,não,1585.3999,1127.1086,1559.4857,1.383616
000139_00,031336,143030,sim,não,não,não,1846.2487,1108.0927,1791.8481,1.617056
000140_00,143030,255789,sim,não,não,não,1190.0806,950.4632,1125.3632,1.184015
000141_00,255788,264853,não,não,não,não,679.3713,525.7540,658.4330,1.252360
000142_00,264853,031336,não,não,não,não,1042.3894,784.4423,1027.3345,1.309637


select: dropped 3 variables (name, highway, qgis_id)

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



osm_id,length_m,geom
<chr>,<dbl>,<LINESTRING [m]>
1000285228,416.696,LINESTRING (325675.3 737227...
1000285229,71.380,LINESTRING (325658.4 737217...
1000285230,83.966,LINESTRING (330455.7 738652...
1000532754,24.138,LINESTRING (330332.6 740090...
1000623723,4.190,LINESTRING (326335.7 738146...
1000623724,3.883,"LINESTRING (326339 7381463,..."


In [7]:
# ------------------------------------------------------------------------------
# Calcular acurácia das rotas modeladas e distâncias percorridas em infra ciclo
# ------------------------------------------------------------------------------


# Abrir rotas modeladas a partir de viagens originais
rotas_m <- sprintf('%s/04_ttmatrix_viagens_modeladas_a_partir_das_originais.csv', pasta_orig_vs_mod)
rotas_m <- read_delim(rotas_m, delim = ';', col_types = 'cccddddcdddd')
rotas_m <- rotas_m %>% select(-c(qgis_id.x, qgis_id.y))

# Juntar com dados das viagens originais
rotas_m <- left_join(ods_vgs, rotas_m, by = 'trip_id')
rotas_m <- rotas_m %>% relocate(c(distance, weight, time, speed), .after = 'qgis_id.y')
head(rotas_m)

select: dropped 2 variables (qgis_id.x, qgis_id.y)

left_join: added 9 columns (distance, weight, time, speed, poly, …)

           > rows only in x         0

           > rows only in y  (      0)

           > matched rows     129,755

           >                 =========

           > rows total       129,755

relocate: columns reordered (trip_id, qgis_id.x, qgis_id.y, distance, weight, …)



trip_id,qgis_id.x,qgis_id.y,distance,weight,time,speed,vg_contramao,vg_loop,vg_exper,vg_parques,dist_total,dist_reta,dist_linha,detour_mm,poly,lon.x,lat.x,lon.y,lat.y
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
000135_00,271157,104168,779.082,252.2742,252.275,11.11761,não,não,não,não,821.4461,562.5676,802.6903,1.426834,jgznCvic|G|@gENYROdNtb@,-46.71660,-23.56869,-46.72109,-23.57161
000138_00,161929,161977,1531.061,485.7503,483.740,11.39418,não,não,não,não,1585.3999,1127.1086,1559.4857,1.383616,`u{nCtu}{GFCZx@Nr@iAl@{BlAtElKdC`FlEhKZv@?`@uHtDuDtB_CdBq@h@_Az@_AdAiBrBy@dAMPEJ,-46.68779,-23.57601,-46.69842,-23.57327
000139_00,031336,143030,1458.244,506.4461,506.265,10.36943,sim,não,não,não,1846.2487,1108.0927,1791.8481,1.617056,bb{nCtw_|GCHM[QoAMsAK?iAH_A@QIIKW}ECWWqEA[Ag@m@{IU_HAcADw@KDIs@E[IFQHYNa@T_B|@yC{GiBmE{C~AOy@,-46.69835,-23.57297,-46.68893,-23.56802
000140_00,143030,255789,1224.437,420.9207,405.094,10.88136,sim,não,não,não,1190.0806,950.4632,1125.3632,1.184015,|bznC`|}{GKk@FQDAp@c@rBuANSt@e@LIFFFENXrBhFdBxDx@dBXv@^QHr@Hr@JEJC`ApCCNhClG~BlFHFF@BDAJ@JdCrF,-46.68880,-23.56799,-46.69571,-23.57371
000141_00,255788,264853,680.669,219.6343,219.633,11.15683,não,não,não,não,679.3713,525.7540,658.4330,1.252360,|f{nCrg_|GkC_GAK@KFGbCmAdJeFLXxGmD,-46.69577,-23.57374,-46.69206,-23.57702
000142_00,264853,031336,1000.576,324.4997,323.348,11.13993,não,não,não,não,1042.3894,784.4423,1027.3345,1.309637,l{{nCjp~{GA?lEhKZv@?`@uHtDuDtB_CdBq@h@_Az@_AdAiBrBy@dAMPQ\a@W,-46.69205,-23.57703,-46.69840,-23.57305


In [19]:
# Listar todos os arquivos de resultados em um dataframe único
# find . -maxdepth 1 -type f  -printf x | wc -c
arqs_resultados <- data.frame(arq = list.files(pasta_rotas_mod, recursive = FALSE, full.names = TRUE))
arqs_resultados <- arqs_resultados %>% mutate(trip_id = str_sub(arq, -22, -14))
# head(arqs_resultados)

# Remover arquivos que já foram executados
rotas_m <- rotas_m %>% filter(!trip_id %in% arqs_resultados$trip_id)
rm(arqs_resultados)
head(rotas_m)

trip_id,qgis_id.x,qgis_id.y,distance,weight,time,speed,vg_contramao,vg_loop,vg_exper,vg_parques,dist_total,dist_reta,dist_linha,detour_mm,poly,lon.x,lat.x,lon.y,lat.y
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


In [18]:
# Inserir cálculos de extensões percorridas em vias comuns ou infra cicloviária
# para cada uma das rotas calculadas
detach("package:tidylog")
# for (tripid in rotas_m$trip_id) {
#   calcular_dist_ciclo(tripid)
# }

# Rodar função para todos os arquivos- multi thread (Jupyter)
(start = Sys.time())
future::plan(future::multicore)
invisible(future.apply::future_lapply(X   = rotas_m$trip_id, 
                                      FUN = calcular_dist_ciclo,
                                      future.seed = TRUE))
Sys.time()
Sys.time() - start

[1] "2024-04-30 20:10:50 -03"

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

...

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Warning message in st_cast.sf(., st_type):
“repeating attributes for all sub-geometries for which they may not be constant”
Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”


In [20]:
# ------------------------------------------------------------------------------
# Juntar todos os resultados
# ------------------------------------------------------------------------------

# Arquivo de saída
out_file <- sprintf('%s/05_ttmatrix_viagens_modeladas_acuracia_infraciclo_carac_viagens.csv', pasta_orig_vs_mod)

# Listar todos os arquivos de resultados em um dataframe único
arqs_resultados <- data.frame(arq = list.files(pasta_rotas_mod, recursive = FALSE, full.names = TRUE))

for (arq in arqs_resultados$arq) {
  # Abrir arquivo de resultados
  arq <- read_delim(arq, delim = ';', col_types = cols(.default = "c"))
   
  # Guardar resultados 
  if (file.exists(out_file)) {
    write_delim(arq, out_file, delim = ';', append = TRUE)
  } else {
    write_delim(arq, out_file, delim = ';', append = FALSE)
  }
  
}


In [22]:
resultados_final <- read_delim(out_file, delim = ';', col_types = cols(.default = "c"))
resultados_final <- resultados_final %>% arrange(trip_id)
head(resultados_final)

trip_id,qgis_id.x,qgis_id.y,distance,weight,time,speed,ext_linhas,acuracia_linhas,pts_intsct,⋯,dist_reta,dist_linha,detour_mm,dist_linha_mod,detour_mod,poly,lon.x,lat.x,lon.y,lat.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
000135_00,271157,104168,779.082,252.27417,252.275,11.117610544049153,802.6903203305397,100,79,⋯,562.5675656565036,802.6903,1.4268335912030026,779.3997,1.3854330529888588,jgznCvic|G|@gENYROdNtb@,-46.716596,-23.56869,-46.721093,-23.571614
000138_00,161929,161977,1531.061,485.750263,483.74,11.394177864141893,284.2080921218263,18.56281964740963,14,⋯,1127.1085865138796,1559.4857,1.3836162004793635,1529.7049,1.357193901548866,`u{nCtu}{GFCZx@Nr@iAl@{BlAtElKdC`FlEhKZv@?`@uHtDuDtB_CdBq@h@_Az@_AdAiBrBy@dAMPEJ,-46.687787,-23.576007,-46.698423,-23.573271
000139_00,031336,143030,1458.244,506.446121,506.265,10.369427868803886,456.0377602718976,31.27307640366753,25,⋯,1108.0927349557112,1791.8481,1.617056085176497,1456.5786,1.3144916071110395,bb{nCtw_|GCHM[QoAMsAK?iAH_A@QIIKW}ECWWqEA[Ag@m@{IU_HAcADw@KDIs@E[IFQHYNa@T_B|@yC{GiBmE{C~AOy@,-46.698347,-23.572974,-46.688926,-23.568022
000140_00,143030,255789,1224.437,420.920685,405.094,10.881358894478812,745.321281548495,60.870529194110844,25,⋯,950.4632374549282,1125.3632,1.1840154943955585,1223.6042,1.2873766725333493,|bznC`|}{GKk@FQDAp@c@rBuANSt@e@LIFFFENXrBhFdBxDx@dBXv@^QHr@Hr@JEJC`ApCCNhClG~BlFHFF@BDAJ@JdCrF,-46.688804,-23.567988,-46.695715,-23.573713
000141_00,255788,264853,680.669,219.634264,219.633,11.15683162366311,658.4329789958946,96.73321085518727,21,⋯,525.7539782014258,658.433,1.2523595204214366,678.8442,1.2911822414017429,|f{nCrg_|GkC_GAK@KFGbCmAdJeFLXxGmD,-46.695772,-23.573744,-46.692057,-23.577017
000142_00,264853,031336,1000.576,324.499665,323.348,11.139928498088747,303.84707642053746,30.36721612556542,25,⋯,784.442330216799,1027.3345,1.3096367450186837,999.5045,1.274159312289743,l{{nCjp~{GA?lEhKZv@?`@uHtDuDtB_CdBq@h@_Az@_AdAiBrBy@dAMPQ\a@W,-46.692052,-23.577026,-46.698396,-23.573045


In [23]:
write_delim(resultados_final, out_file, delim = ';')